# Utils

In [1]:
from scripts import utils
from scripts import metrics
from scripts import inference_model
from tqdm.notebook import tqdm

In [2]:
DATASET_NAME = 'rcp-meetings/rudialogsum_v2'

## MTSAIR_Cotype_Nano_inference

In [13]:
MODEL_NAME = "MTSAIR/Cotype-Nano"

In [4]:
data = utils.get_dataset(DATASET_NAME)['train']

In [5]:
pipe = utils.get_model_pipeline(MODEL_NAME)

### zero_shot

In [ ]:
infer_dict = {}

for idx, text in tqdm(enumerate(data)):
    messages = [
        {"role": "system", "content": "Реши задачу по инструкции ниже. Не давай никаких объяснений и пояснений к своему ответу. Инструкция: суммаризируй диалог до одного двух предложений."},
        {"role": "user", "content": text['dialog']}
        ]
    pred = inference_model.get_predicts_hf(pipe, messages)
    infer_dict[idx] = pred

In [8]:
utils.save_json(infer_dict, './DATA/MTSAIR_Cotype_Nano_inference/pred_zero_shot.json')

### one_shot

In [ ]:
example_indices_full = [30]

infer_dict = {}

for idx, text in tqdm(enumerate(data)):
    one_shot_prompt = utils.make_prompt(data, example_indices_full, idx)
    messages = [
        {"role": "system", "content": "Ты ИИ-помощник. Инструкция: суммаризируй диалог до одного двух предложений."},
        {"role": "user", "content": one_shot_prompt}
        ]
    pred = inference_model.get_predicts_hf(pipe, messages)
    infer_dict[idx] = pred

0it [00:00, ?it/s]

In [10]:
utils.save_json(infer_dict, './DATA/MTSAIR_Cotype_Nano_inference/pred_one_shot.json')

## T_lite_it_1.0_Q8_0_GGUF

In [3]:
MODEL_NAME = "hf.co/t-tech/T-lite-it-1.0-Q8_0-GGUF:Q8_0"

In [4]:
data = utils.get_dataset(DATASET_NAME)['train']

### one_shot

In [5]:
example_indices_full = [30]

infer_dict = {}

for idx, text in tqdm(enumerate(data)):
    one_shot_prompt = utils.make_prompt(data, example_indices_full, idx)
    messages = [
        {"role": "system", "content": "Ты ИИ-помощник. Тебе даны два диалога, первый написан как пример, а второй нужно сократить до двух предложений. Не пиши ничего лишнего, только сокращенный текст."},
        {"role": "user", "content": one_shot_prompt}
        ]
    pred = inference_model.get_predicts_ollama(MODEL_NAME, messages)
    infer_dict[idx] = pred

0it [00:00, ?it/s]

In [6]:
utils.save_json(infer_dict, './DATA/T_lite_it_1.0_Q8_0_GGUF/pred_one_shot.json')

## T_pro_it_1.0_Q4_K_M_GGUF

In [ ]:
MODEL_NAME = "hf.co/t-tech/T-pro-it-1.0-Q4_K_M-GGUF:latest"

In [ ]:
data = utils.get_dataset(DATASET_NAME)['train']

### one shot

In [ ]:
example_indices_full = [30]

infer_dict = {}

for idx, text in tqdm(enumerate(data)):
    one_shot_prompt = utils.make_prompt(data, example_indices_full, idx)
    messages = [
        {"role": "system", "content": "Ты ИИ-помощник. Тебе даны два диалога, первый написан как пример, а второй нужно сократить до двух предложений. Не пиши ничего лишнего, только сокращенный текст."},
        {"role": "user", "content": one_shot_prompt}
        ]
    pred = inference_model.get_predicts_ollama(MODEL_NAME, messages)
    infer_dict[idx] = pred

In [ ]:
utils.save_json(infer_dict, './DATA/T_lite_it_1.0_Q8_0_GGUF/pred_one_shot.json')

# Metrics

In [32]:
data = utils.get_dataset(DATASET_NAME)['train']
preds = utils.load_json('./DATA/T_pro_it_1.0_Q4_K_M_GGUF/pred_one_shot.json')
preds = [preds[str(idx)] for idx in range(len(preds))]

In [26]:
metrics_model = metrics.get_metrics(preds, data['summary'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wzakh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\wzakh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wzakh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Computing metrics...
Rouge computed
Bleu computed
Meteor computed


c:\Programs\Anaconda\envs\NLP\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Bertscore computed
Metrics computed


In [27]:
metrics_model

{'rouge1': 0.5692213597093654,
 'rouge2': 0.47515858017750734,
 'rougeL': 0.5300113575545231,
 'rougeLsum': 0.5299346954143009,
 'bleu': 0.20037898308333082,
 'meteor': 0.3095146176260515,
 'bertscore': 0.7710646254675729}

In [16]:
utils.save_json(metrics_model, './DATA/T_pro_it_1.0_Q4_K_M_GGUF/metrics_one_shot.json')

# DATASET inference

In [3]:
from ollama import chat
from ollama import ChatResponse

In [4]:
MODEL_NAME = "hf.co/t-tech/T-lite-it-1.0-Q8_0-GGUF:Q8_0"
# MODEL_NAME = "hf.co/t-tech/T-pro-it-1.0-Q4_K_M-GGUF:latest"

In [5]:
data = utils.load_json('./DATASET/DATASET_DIALOG_SUMMARY.json')

In [6]:
example_indices_full = [0]

infer_dict = {}

options = {'temperature': 0.1, 
           'repeat_penalty ': 1.05, 
           'num_predict': 128, 
        #    'repeat_last_n':-1, 
           'top_k': 70, 
           'top_p': 0.8,
        #    'mirostat_tau': 1.0,
        #    'mirostat': 2,
           'num_ctx': 16384
            }

for idx, text in tqdm(enumerate(range(len(data['dialog'])))):
    # if idx in example_indices_full:
    #     continue
    one_shot_prompt = utils.make_prompt_final(data, example_indices_full, idx)
    messages = [
        {"role": "system", "content": "Ты ИИ-помощник. Тебе дан пример сокращенного диалога, тебе нужно сократить по такой же схеме диалог. Перепишите текст, сокращая его до ключевых задач и сроков, без добавления лишних комментариев или промежуточных вариантов. Только итоговые задачи и сроки."},
        {"role": "user", "content": one_shot_prompt}
        ]
    res: ChatResponse = chat(MODEL_NAME, messages, options=options)
    pred = res['message']['content']
    # pred = inference_model.get_predicts_ollama(
    #     MODEL_NAME, 
    #     message=messages,
    #     options=options)
    infer_dict[idx] = pred
    print(10*'-', idx, 10*'-')
    print(pred)
    # !ollama stop hf.co/t-tech/T-lite-it-1.0-Q8_0-GGUF:Q8_0

0it [00:00, ?it/s]

---------- 0 ----------
- **Мария**, Завершить фильтры дашборда и визуализацию прогноза продаж, **Фильтры — до завтра, Визуализация — до пятницы**.  
- **Дмитрий**, Оптимизировать модель с добавлением синтетических данных и передать данные Марии, **Конец среды**.  
- **Екатерина**, Уточнить сегменты клиентской базы и подготовить гипотезы для таргетинга
---------- 1 ----------
- **Анна (Специалист по обработке данных)**, Перепроверить данные по складу Новосибирска и передать Ивану для ETL, до конца четверга.
- **Иван (Разработчик ETL-пайплайнов)**, Завершить ETL-пайплайн с обработкой данных из мобильного приложения, до пятницы. Передать Людмиле.
- **Людмила (Аналитик BI)**, Готовить дашборд для


In [7]:
infer_dict['prompt'] = messages
infer_dict['options'] = options

In [8]:
utils.save_json(infer_dict, './DATASET/predicts/pred_one_shot_prompt_6.json')

## Metrics for dataset

In [9]:
data = utils.load_json('./DATASET/DATASET_DIALOG_SUMMARY.json')
preds = utils.load_json('./DATASET/predicts/pred_one_shot_prompt_6.json')
preds = [preds[str(idx)] for idx in range(len(data['summary']))]

In [10]:
metrics_model = metrics.get_metrics(preds, data['summary'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wzakh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\wzakh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wzakh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Computing metrics...
Rouge computed
Bleu computed
Meteor computed


c:\Programs\Anaconda\envs\NLP\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Bertscore computed
Metrics computed


In [11]:
metrics_model

{'rouge1': 0.32411255411255413,
 'rouge2': 0.08787878787878789,
 'rougeL': 0.3268831168831169,
 'rougeLsum': 0.32961038961038963,
 'bleu': 0.26305508598218086,
 'meteor': 0.47417791720898667,
 'bertscore': 0.7938736796379089}

In [ ]:
utils.save_json(metrics_model, './DATASET/scores')